In [4]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
!playwright install

In [96]:
SEASONS = list(range(2023,2025))

In [97]:
SEASONS

[2023, 2024]

In [98]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [99]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [100]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    if html is None:
        print(f"Failed to retrieve HTML for season {season}")
        return
        
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")

        if html is None:
            print(f"Failed to retrieve HTML for season {season}")
            return
        
        with open(save_path, "w+") as f:
            f.write(html)

In [102]:
for season in SEASONS: 
    await scrape_season(season)

Timeout error on https://www.basketball-reference.com/leagues/NBA_2023_games.html
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com


In [103]:
standings_files = os.listdir(STANDINGS_DIR)

In [104]:
standings_files

['NBA_2024_games-may.html',
 'NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2023_games-february.html',
 'NBA_2024_games-february.html',
 'NBA_2024_games-january.html',
 '.ipynb_checkpoints',
 'NBA_2024_games-april.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html',
 'NBA_2024_games-june.html']

In [105]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [111]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Thunder vs Nets, January 15, 2023 | Basketball-Reference.com
Pistons vs Bucks, November 2, 2022 | Basketball-Reference.com


In [107]:
standings_files = [s for s in standings_files if ".html" in s]

In [108]:
standings_files

['NBA_2024_games-may.html',
 'NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2023_games-february.html',
 'NBA_2024_games-february.html',
 'NBA_2024_games-january.html',
 'NBA_2024_games-april.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html',
 'NBA_2024_games-june.html']